
<h1 id="Вступительное-задание-в-комманду-ML.">Вступительное задание в комманду ML.<a class="anchor-link" href="#Вступительное-задание-в-комманду-ML.">¶</a></h1><p>Рябушев Антон. Май 2019 года.</p>



<h1 id="Постановка-задачи">Постановка задачи<a class="anchor-link" href="#Постановка-задачи">¶</a></h1>



<p>Реализуйте максимально эффективным образом алгоритм иерархической кластеризации с алгоритмом объединения single-link clustering для точек, расположенных на прямой.</p>



<h1 id="Введение">Введение<a class="anchor-link" href="#Введение">¶</a></h1>



<p>Было реализовано два алгоритма:</p>
<ul>
<li>Наивный</li>
<li>Быстрый</li>
</ul>
<p>Ниже представлено описание каждого из алгоритмов.</p>



<h1 id="Общая-часть">Общая часть<a class="anchor-link" href="#Общая-часть">¶</a></h1>



<p>Node - вершина дерева поддерживающая следующий инвариант.</p>
<p>[l, r) - полуинтервал данных, который доступен из данной вершины.</p>


In [1]:
%matplotlib inline

from scipy.cluster import hierarchy
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
class Node:
    _counter = 0
    def __init__(self, l, r, childs=None):
        self._l, self._r, self._childs = l, r, childs
        self._id = Node._counter
        Node._counter += 1

    def __str__(self, deep=0):
        prefix = "|--" * deep
        # result = "{}{}, l: {}, r: {}".format(prefix, self._id, self._l, self._r)
        result = "{}{}".format(prefix, self._id, self._l, self._r)
        if self._childs:
            for child in self._childs:
                result += '\n' + child.__str__(deep + 1) 
        return result

    def __eq__(self, other):
        result = self._l == other._l and self._r == other._r
        if self.leaf() or other.leaf():
            return result and self.leaf() == other.leaf()
        for first, second in zip(self._childs, other._childs):
            result = result and first == second
        return result

    def leaf(self):
        return self._childs is None

In [3]:
def get_deltas(value):
    deltas = [right - left for left, right in zip(value[:-1], value[1:])]
    return deltas


<h1 id="The-naive-algorithm">The naive algorithm<a class="anchor-link" href="#The-naive-algorithm">¶</a></h1>



<p>Рекурсивное построение дерева.</p>
<ul>
<li>Найдем самое большое расстояние между соседнимим элементами, их может быть несколько одинаковых.</li>
<li>Объединение по данному расстоянию очевидным образом будет происходить последним. </li>
<li>Разделим вершины на разные классы в зависимости расположения каждой вершины от данных промежутков. </li>
<li>Построим дерево для каждого множества вершин и подвесим их за одну общую вершину.</li>
</ul>
<p>Алгоритм похож на алгоритм quicksort. В худшем случае получим $ O(n^2)$</p>


In [4]:
def recursion_build(deltas, start_index):
    if not deltas:
        return Node(start_index, start_index + 1)
    max_delta = max(deltas)
    childs = list()
    new_deltas = list()
    start = 0
    for index, delta in enumerate(deltas):
        if delta == max_delta:
            childs.append(recursion_build(new_deltas, start_index + start))
            new_deltas = list()
            start = index + 1
        else:
            new_deltas.append(delta)
    childs.append(recursion_build(new_deltas, start_index + start))
    return Node(start_index, start_index + len(deltas) + 1, childs)


def build_tree_simple(value):
    deltas = get_deltas(value)
    return recursion_build(deltas, 0)


<h1 id="The-smart-algorithm">The smart algorithm<a class="anchor-link" href="#The-smart-algorithm">¶</a></h1>



<ul>
<li>Будем строить дерево снизу вверх, начиная от самых маленьких расстояний.</li>
<li>После обработки каждого расстояния будем поддерживать следующий инвариант. Для каждого кластера, current_nodes от граничных точек этого кластера указывают на него.</li>
<li>Для примера до начала работы алгоритма, каждый элемент current_nodes указывает на нужный кластер $[i, i + 1)$. После завершения работы алгоритма, ячейки 0, n-1 будут ссылаться на вершину отвечающую за кластер $[0, n)$, то-есть за все данные.</li>
<li>При фиксированном расстоянии d будем идти от меньших по индексам вершин к большим, у которых следующая вершина на расстоянии d. </li>
<li>Пока они соседнии, кладем их в список.</li>
<li>Когда следующая в обходе вершина не соседня, создаем новый кластер.</li>
<li>При создании кластера изменим current_nodes для границ, тем самым сохранив инвариант.</li>
</ul>
<p>Асимптотика $ O(n \log n) $</p>


In [5]:
def check_neighborhood_of_vertices(first, second, current_nodes):
    return current_nodes[first]._r == current_nodes[second]._l


def union_group_to_new_node(group, current_nodes):
    childs = list(map(lambda index: current_nodes[index], group))
    l, r = childs[0]._l, childs[-1]._r
    new_node = Node(l, r, childs)
    current_nodes[l] = new_node
    current_nodes[r - 1] = new_node


def build_tree_smart(value):
    deltas = get_deltas(value)
    current_nodes = [Node(index, index + 1) for index in range(len(value))]
    delta_to_indexes = dict()
    for index, delta in enumerate(deltas):
        if delta in delta_to_indexes:
            delta_to_indexes[delta].append(index)
        else:
            delta_to_indexes[delta] = [index]
    for delta, indexes in sorted(delta_to_indexes.items()):
        group = list()
        for index in indexes:
            if group and not check_neighborhood_of_vertices(group[-1], index, current_nodes):
                group.append(group[-1] + 1)
                union_group_to_new_node(group, current_nodes)
                group.clear()
            group.append(index)
        if group:
            group.append(group[-1] + 1)
            union_group_to_new_node(group, current_nodes)
    return current_nodes[0]

# Test

In [6]:
def test(value):
    assert value.sort() != value
    first = build_tree_simple(value)
    second = build_tree_smart(value)
    assert first == second

def test_all_eq():
    values = list(range(1, 100, 5))
    test(values)

def some_tests():
    test([2 ** i for i in range(10)])
    test([0, 1, 3, 4])
    test_all_eq()


def random_test(num_of_element, max_int):
    values = sorted([random.randrange(max_int) for i in range(num_of_element)])
    test(values)


def n_random_test(num_of_test=100, num_of_element=100, max_int=100):
    for i in range(num_of_test):
        random_test(num_of_element, max_int)


<h1 id="Run">Run<a class="anchor-link" href="#Run">¶</a></h1>


In [7]:
random.seed(42)
some_tests()
n_random_test(10000, 100, 10)


<h1 id="Some-examples">Some examples<a class="anchor-link" href="#Some-examples">¶</a></h1>


Выводятся индексы в исходном массиве, а не сами данные.

In [8]:
Node._counter = 0
print(build_tree_smart([1, 1, 1]))

3
|--0
|--1
|--2


In [9]:
Node._counter = 0
print(build_tree_smart([1, 2, 4]))

4
|--3
|--|--0
|--|--1
|--2


In [10]:
Node._counter = 0
print(build_tree_smart([1, 3, 4]))

4
|--0
|--3
|--|--1
|--|--2



<h1 id="Bad-cases">Bad cases<a class="anchor-link" href="#Bad-cases">¶</a></h1>



<p>Ясно, что когда все расстояния равны, кластеризации совсем не будет.</p>


In [11]:
Node._counter = 0
print(build_tree_smart(range(1, 20, 4)))

5
|--0
|--1
|--2
|--3
|--4



<p>Рассмортим точку у которой слева много точек, а справа одна, но чуть ближе. 
Так как точек слева больше, хочется сказать, что скорее всего, она принадлежит левому классу.</p>


In [12]:
Node._counter = 0
print(build_tree_smart([0, 0, 0, 0, 0, 0, 500, 999]))

10
|--8
|--|--0
|--|--1
|--|--2
|--|--3
|--|--4
|--|--5
|--9
|--|--6
|--|--7



<p>Рассмортим точку у которой слева много близких друг к другу точек на расстояние d, а справа несколько подряд идущих точек с расстоянием d - eps. Мы объединили точку с самой правой, которая заметно дальше от нашей точки, чем точки слева.</p>


In [13]:
Node._counter = 0
print(build_tree_smart([0, 0, 0, 0, 0, 0, 2, 3, 4, 5]))

12
|--10
|--|--0
|--|--1
|--|--2
|--|--3
|--|--4
|--|--5
|--11
|--|--6
|--|--7
|--|--8
|--|--9
